# Access Control

<a href="http://35.236.121.59/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fproject-chip%2Fconnectedhomeip&urlpath=lab%2Ftree%2Fconnectedhomeip%2Fdocs%2Fguides%2Frepl%2FMatter%2520-%2520Access%2520Control.ipynb&branch=master">
<img src="https://i.ibb.co/hR3yWsC/launch-playground.png" alt="drawing" width="130"/>
</a>
<br></br>

This document explains how to use Access Control in Matter, and will be updated as development proceeds.

## What Does and Doesn’t Work Right Now?

Briefly, you can read and write the entire ACL attribute in the all-clusters-app, but Access Control isn’t yet turned on, so it won’t affect interactions. There’s almost no error checking when writing the ACL attribute (e.g. ensuring subjects match auth mode, only your fabric can be written, etc.) so exercise caution for now.

## Clear Persisted Storage

Let's clear out our persisted storage (if one exists) to start from a clean slate.

In [1]:
import os, subprocess

if os.path.isfile('/tmp/repl-storage.json'):
    os.remove('/tmp/repl-storage.json')

# So that the all-clusters-app won't boot with stale prior state.
os.system('rm -rf /tmp/chip_*')

0

## Initialization

Let's first begin by setting up by importing some key modules that are needed to make it easier for us to interact with the Matter stack.

> **NOTE**: _This is not needed if you launch the REPL from the command-line._

In [2]:
%reset -f
import importlib.util
spec = importlib.util.find_spec('chip.ChipReplStartup')
%run {spec.origin}

 Replacing  store path ./credentials/development/paa-root-certs with 
/home/root/repos/connectedhomeip/credentials/development/paa-root-certs
Note that you are still running from 
/home/root/repos/connectedhomeip/docs/development_controllers/chip-repl so other relative
paths may be off.

[1738153365.139353][2283552:2283552] CHIP:CTL: Setting attestation nonce to random value
[1738153365.139578][2283552:2283552] CHIP:CTL: Setting CSR nonce to random value
[1738153365.151683][2283552:2283552] CHIP:DL: NVS set: chip-factory/unique-id = "107DC5BE4DC63F48"
[1738153365.152834][2283552:2283552] CHIP:DL: NVS set: chip-factory/vendor-id = 65521 (0xFFF1)
[1738153365.154302][2283552:2283552] CHIP:DL: NVS set: chip-factory/product-id = 32769 (0x8001)
[1738153365.155697][2283552:2283552] CHIP:DL: NVS set: chip-counters/reboot-count = 1 (0x1)
[1738153365.157205][2283552:2283552] CHIP:DL: NVS set: chip-counters/total-operational-hours = 0 (0x0)
[1738153365.158744][2283552:2283552] CHIP:DL: NVS set: chip-counters/boot-reason = 0 (0x0)
[1738153365.160171][2283552:2283552] CHIP:DL: NVS set: chip-config/regulatory-location = 0 (0x0)
[1738153365.161859][2283552:2283552] CHIP:DL: NVS set: chip-config/location-capability = 2 (0x2)
[1738153365.162176][2283552:2283552] CHIP:DL: Got Ethernet i

─────────────────────────────────────────────────── Matter REPL ───────────────────────────────────────────────────

            Welcome to the Matter Python REPL!

            For help, please type matterhelp()

            To get more information on a particular object/class, you can pass
            that into matterhelp() as well.

            

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

2025-01-29 13:22:45 ThinkPad chip.storage[2283552] ERROR [Errno 2] No such file or directory: '/tmp/repl-storage.json'
2025-01-29 13:22:45 ThinkPad chip.storage[2283552] CRITICAL Could not load configuration from /tmp/repl-storage.json - resetting configuration...
2025-01-29 13:22:45 ThinkPad chip.storage[2283552] WARNING No valid SDK configuration present - clearing out configuration
2025-01-29 13:22:45 ThinkPad chip.storage[2283552] WARNING No valid REPL configuration present - clearing out configuration


The following objects have been created:

certificateAuthorityManager:    Manages a list of CertificateAuthority instances.
        caList:                         The list of CertificateAuthority instances.
        caList[n].adminList[m]:         A specific FabricAdmin object at index m for the nth CertificateAuthority 
instance.

Default CHIP Device Controller (NodeId: 112233): has been initialized to manage caList[0].adminList[0] (FabricId = 
1), and is available as devCtrl

## Commission and Setup Server

### Launch Server

Let's launch an instance of the `chip-all-clusters-app`.

In [3]:
import time, os
import subprocess
os.system('pkill -f chip-all-clusters-app')
time.sleep(1)

CI_APP_PATH = '../../../out/linux-x64-all-clusters-ipv6only-no-ble-no-wifi-clang-test/chip-all-clusters-app'
LOCAL_APP_PATH = '../../../out/linux-x64-all-clusters/chip-all-clusters-app'

# Check if the app built by GitHub Actions is present. Otherwise use the standard all-clusters-app 
if (os.path.isfile(CI_APP_PATH)):
    appPath = CI_APP_PATH
else:
    appPath = LOCAL_APP_PATH

process = subprocess.Popen(appPath, stdout=subprocess.DEVNULL)
time.sleep(1)

### Commission Target

Commission the target with a NodeId of 2.

In [4]:
await devCtrl.CommissionOnNetwork(2, 20202021)

2

## Bootstrap ACLs

During Commissioning, an ACL that assigns Administer rights to the commissioner(i.e. chip-repl) was automatically installed on the commissionee.

In [5]:
await devCtrl.ReadAttribute(2, [ (0, Clusters.OperationalCredentials)], returnClusterObject=True)


{
│   0: {
│   │   <class 'chip.clusters.Objects.OperationalCredentials'>: OperationalCredentials(
│   │   │   (data version)=3950260077,
│   │   │   NOCs=[
│   │   │   │   NOCStruct(
│   │   │   │   │   noc=b'\x150\x01\x01\x01$\x02\x017\x03$\x13\x02\x18&\x04\x80"\x81\'&\x05\x80%M:7\x06$\x15\x01$\x11\x02\x18$\x07\x01$\x08\x010\tA\x04\xbe1\xb2Q\xd2\xc2\x1fn\x9b\xc1\xe5}\x188\xca\x08B\x05MOu\xd0\xb4\xbb\x9d\xeaS\xa6\xb0\xb7\xc8\x90\xd2\x0b\xc2I\xa1\x1e\xee\x92\xa9P\x02\xc2\xe8~p\x07\xb7\x86&\x01\x15\xcdl\x838\x8f\x12s_\xd6\x8d77\n5\x01(\x01\x18$\x02\x016\x03\x04\x02\x04\x01\x180\x04\x14D\x99T\xc8\\Q\xb4\x10cD\x9dC>n"5\xf5<Q|0\x05\x14\xc9\x9b\xd7\xa7d0k\x05\xaf\xd8W\x91\x982\x19\xb9\xed\xda\x7f \x180\x0b@\xfd\xb0v_\x1f\x8c\xc8\xa0\xb3b\n\x8c\x13X\x1f\xf0*\xf7\xd2\x93\xaaN\xbd/\xa6\x99\xc5\xa2\\\xcb\xbd\xcb\x8e\xfbD\xd5\xdd\xd1\xc7\xe1H\xd9\x8c\xefEU\xccG\x92!\x94\xd1:P\x0eqU\xb9\x94\x9e6\x96z\x1c\x18',
│   │   │   │   │   icac=b'\x150\x01\x01\x01$\x02\x017\x03$\x14\x01\x18&\x04\x80"\x81\

### Automatically Installed ACL

In [6]:
data = await devCtrl.ReadAttribute(2, [ (0, Clusters.AccessControl.Attributes.Acl) ] )
data


{
│   0: {
│   │   <class 'chip.clusters.Objects.AccessControl'>: {
│   │   │   <class 'chip.clusters.Attribute.DataVersion'>: 318338779,
│   │   │   <class 'chip.clusters.Objects.AccessControl.Attributes.Acl'>: [
│   │   │   │   AccessControlEntryStruct(
│   │   │   │   │   privilege=<AccessControlEntryPrivilegeEnum.kAdminister: 5>,
│   │   │   │   │   authMode=<AccessControlEntryAuthModeEnum.kCase: 2>,
│   │   │   │   │   subjects=[
│   │   │   │   │   │   112233
│   │   │   │   │   ],
│   │   │   │   │   targets=Null,
│   │   │   │   │   fabricIndex=1
│   │   │   │   )
│   │   │   ]
│   │   }
│   }
}

In [7]:
acl = data[0][chip.clusters.Objects.AccessControl][chip.clusters.Objects.AccessControl.Attributes.Acl]
acl


[
│   AccessControlEntryStruct(
│   │   privilege=<AccessControlEntryPrivilegeEnum.kAdminister: 5>,
│   │   authMode=<AccessControlEntryAuthModeEnum.kCase: 2>,
│   │   subjects=[
│   │   │   112233
│   │   ],
│   │   targets=Null,
│   │   fabricIndex=1
│   )
]

### Installing a CASE ACL

In [8]:
acl.append(Clusters.AccessControl.Structs.AccessControlEntryStruct(
    fabricIndex = 1,
    privilege = Clusters.AccessControl.Enums.AccessControlEntryPrivilegeEnum.kOperate,
    authMode = Clusters.AccessControl.Enums.AccessControlEntryAuthModeEnum.kCase,
    targets = [ Clusters.AccessControl.Structs.AccessControlTargetStruct(
        endpoint = 1,
    ) ] ) )
acl


[
│   AccessControlEntryStruct(
│   │   privilege=<AccessControlEntryPrivilegeEnum.kAdminister: 5>,
│   │   authMode=<AccessControlEntryAuthModeEnum.kCase: 2>,
│   │   subjects=[
│   │   │   112233
│   │   ],
│   │   targets=Null,
│   │   fabricIndex=1
│   ),
│   AccessControlEntryStruct(
│   │   privilege=<AccessControlEntryPrivilegeEnum.kOperate: 3>,
│   │   authMode=<AccessControlEntryAuthModeEnum.kCase: 2>,
│   │   subjects=Null,
│   │   targets=[
│   │   │   AccessControlTargetStruct(
│   │   │   │   cluster=Null,
│   │   │   │   endpoint=1,
│   │   │   │   deviceType=Null
│   │   │   )
│   │   ],
│   │   fabricIndex=1
│   )
]

In [9]:
await devCtrl.WriteAttribute(2, [ (0, Clusters.AccessControl.Attributes.Acl( acl ) ) ] )


[
│   AttributeStatus(
│   │   Path=AttributePath(
│   │   │   EndpointId=0,
│   │   │   ClusterId=31,
│   │   │   AttributeId=0
│   │   ),
│   │   Status=<Status.Success: 0>
│   )
]

In [10]:
await devCtrl.ReadAttribute(2, [ (0, Clusters.AccessControl.Attributes.Acl ) ] )


{
│   0: {
│   │   <class 'chip.clusters.Objects.AccessControl'>: {
│   │   │   <class 'chip.clusters.Attribute.DataVersion'>: 318338782,
│   │   │   <class 'chip.clusters.Objects.AccessControl.Attributes.Acl'>: [
│   │   │   │   AccessControlEntryStruct(
│   │   │   │   │   privilege=<AccessControlEntryPrivilegeEnum.kAdminister: 5>,
│   │   │   │   │   authMode=<AccessControlEntryAuthModeEnum.kCase: 2>,
│   │   │   │   │   subjects=[
│   │   │   │   │   │   112233
│   │   │   │   │   ],
│   │   │   │   │   targets=Null,
│   │   │   │   │   fabricIndex=1
│   │   │   │   ),
│   │   │   │   AccessControlEntryStruct(
│   │   │   │   │   privilege=<AccessControlEntryPrivilegeEnum.kOperate: 3>,
│   │   │   │   │   authMode=<AccessControlEntryAuthModeEnum.kCase: 2>,
│   │   │   │   │   subjects=Null,
│   │   │   │   │   targets=[
│   │   │   │   │   │   AccessControlTargetStruct(
│   │   │   │   │   │   │   cluster=Null,
│   │   │   │   │   │   │   endpoint=1,
│   │   │   │   │   │   │   de

### Installing a Group ACL

In [11]:
acl.append( Clusters.AccessControl.Structs.AccessControlEntryStruct(
    privilege = Clusters.AccessControl.Enums.AccessControlEntryPrivilegeEnum.kManage,
    authMode = Clusters.AccessControl.Enums.AccessControlEntryAuthModeEnum.kGroup,
    subjects = [ 123, 456 ],
    targets = [
      Clusters.AccessControl.Structs.AccessControlTargetStruct(
        cluster = Clusters.OnOff.id,
      ),
      Clusters.AccessControl.Structs.AccessControlTargetStruct(
        endpoint = 1,
      ),
      Clusters.AccessControl.Structs.AccessControlTargetStruct(
        cluster = Clusters.LevelControl.id,
        endpoint = 2,
      ) ] ) )
acl


[
│   AccessControlEntryStruct(
│   │   privilege=<AccessControlEntryPrivilegeEnum.kAdminister: 5>,
│   │   authMode=<AccessControlEntryAuthModeEnum.kCase: 2>,
│   │   subjects=[
│   │   │   112233
│   │   ],
│   │   targets=Null,
│   │   fabricIndex=1
│   ),
│   AccessControlEntryStruct(
│   │   privilege=<AccessControlEntryPrivilegeEnum.kOperate: 3>,
│   │   authMode=<AccessControlEntryAuthModeEnum.kCase: 2>,
│   │   subjects=Null,
│   │   targets=[
│   │   │   AccessControlTargetStruct(
│   │   │   │   cluster=Null,
│   │   │   │   endpoint=1,
│   │   │   │   deviceType=Null
│   │   │   )
│   │   ],
│   │   fabricIndex=1
│   ),
│   AccessControlEntryStruct(
│   │   privilege=<AccessControlEntryPrivilegeEnum.kManage: 4>,
│   │   authMode=<AccessControlEntryAuthModeEnum.kGroup: 3>,
│   │   subjects=[
│   │   │   123,
│   │   │   456
│   │   ],
│   │   targets=[
│   │   │   AccessControlTargetStruct(
│   │   │   │   cluster=6,
│   │   │   │   endpoint=Null,
│   │   │   │   deviceType=

In [12]:
await devCtrl.WriteAttribute(2, [ (0, Clusters.AccessControl.Attributes.Acl( acl ) ) ] ) 


[
│   AttributeStatus(
│   │   Path=AttributePath(
│   │   │   EndpointId=0,
│   │   │   ClusterId=31,
│   │   │   AttributeId=0
│   │   ),
│   │   Status=<Status.Success: 0>
│   )
]

In [13]:
await devCtrl.ReadAttribute(2, [ (0, Clusters.AccessControl.Attributes.Acl ) ] )


{
│   0: {
│   │   <class 'chip.clusters.Objects.AccessControl'>: {
│   │   │   <class 'chip.clusters.Attribute.DataVersion'>: 318338786,
│   │   │   <class 'chip.clusters.Objects.AccessControl.Attributes.Acl'>: [
│   │   │   │   AccessControlEntryStruct(
│   │   │   │   │   privilege=<AccessControlEntryPrivilegeEnum.kAdminister: 5>,
│   │   │   │   │   authMode=<AccessControlEntryAuthModeEnum.kCase: 2>,
│   │   │   │   │   subjects=[
│   │   │   │   │   │   112233
│   │   │   │   │   ],
│   │   │   │   │   targets=Null,
│   │   │   │   │   fabricIndex=1
│   │   │   │   ),
│   │   │   │   AccessControlEntryStruct(
│   │   │   │   │   privilege=<AccessControlEntryPrivilegeEnum.kOperate: 3>,
│   │   │   │   │   authMode=<AccessControlEntryAuthModeEnum.kCase: 2>,
│   │   │   │   │   subjects=Null,
│   │   │   │   │   targets=[
│   │   │   │   │   │   AccessControlTargetStruct(
│   │   │   │   │   │   │   cluster=Null,
│   │   │   │   │   │   │   endpoint=1,
│   │   │   │   │   │   │   de